In [381]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from PIL import Image

In [144]:
data_rent = pd.read_csv('cso_rtb_avg_month_rent.csv')
counties = {'Donegal', 'Leitrim', 'Sligo', 'Dublin', 'Kildare', 'Meath', 'Wicklow', 'Laois', 'Longford', 'Offaly', 'Westmeath', 'Clare', 'Limerick', 'Tipperary', 'Cavan', 'Louth', 'Monaghan', 'Carlow', 'Kilkenny', 'Waterford', 'Wexford', 'Cork', 'Kerry', 'Galway', 'Mayo', 'Roscommon'}

['Donegal', 'Leitrim', 'Sligo', 'Dublin', 'Kildare', 'Meath', 'Wicklow', 'Laois', 'Longford', 'Offaly', 'Westmeath', 'Clare', 'Limerick', 'Tipperary', 'Cavan', 'Louth', 'Monaghan', 'Carlow', 'Kilkenny', 'Waterford', 'Wexford', 'Cork', 'Kerry', 'Galway', 'Mayo', 'Roscommon']


In [267]:
# Source https://community.plotly.com/t/group-each-graph-dimension-in-the-legend-of-plotly/58118
def add_grouped_legend(fig, data_frame, x, graph_dimensions):
	"""Create a grouped legend based on the example here https://stackoverflow.com/a/69829305/8849755
	- fig: The figure in which to add such grouped legend.
	- data_frame: The data frame from which to create the legend, in principle it should be the same that was plotted in `fig`.
	- graph_dimensions: A dictionary with the arguments such as `color`, `symbol`, `line_dash` passed to plotly.express functions you want to group, with the names of the columns in the data_frame."""
	param_list = [{'px': {dimension: dimension_value}, 'lg': {'legendgrouptitle_text': dimension_value}} for dimension, dimension_value in graph_dimensions.items()]
	legend_traces = []
	for param in param_list:
		this_dimension_trace = px.line(
			data_frame,
			x = x,
			y = [float('NaN') for i in range(len(data_frame))],
			**param["px"],
		).update_traces(
			# **param["lg"],
			# legendgroup = str(param["px"]),
		)
		if 'color' not in param['px']:
			this_dimension_trace.update_traces(
				marker = {'color': '#000000'},
				line = {'color': '#000000'},
			)
		legend_traces.append(this_dimension_trace)
	for t in legend_traces:
		fig.add_traces(t.data)

In [21]:
max_rent = 2635.18
min_rent = 419.55

graph_height = 500
graph_width = 700

legend = Image.open("legend.png")

for county in counties:
    df = data_rent[data_rent['Location'] == county]

    fig = px.line(df, x="Quarter", y="VALUE", color='Property Type', title="RTB Average Monthly Rent (€) - {}".format(county), width=graph_width, height=graph_height)

    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            # tickvals = [2019.25, 2019.50, 2019.75, 2019.99, 2020.25, 2020.50, 2020.75, 2020.99, 2021.25, 2021.50, 2021.75, 2021.99, 2022.25, 2022.50,],
            # ticktext = ['2019 Q1', '2019 Q2', '2019 Q3', '2019 Q4', '2020 Q1', '2020 Q2', '2020 Q3', '2020 Q4', '2021 Q1', '2021 Q2', '2021 Q3', '2021 Q4', '2022 Q1', '2022 Q2'],
            tickangle = -45
        ),
        # showlegend = False,
        margin=dict(l=0, r=0, t=50),
        font=dict(
            size=16,  # Set the font size here
        ),
        xaxis_title="Year (by quarter)", yaxis_title="Monthly Rent (€)",
        legend=dict(
            orientation="h",
            y=-0.3
        )
    )
    
    # fig.write_html('images/{}.html'.format(county), include_plotlyjs=False, full_html=False)
    
for countyA in { 'Dublin'}:
    for countyB in {'Wexford'}:
        if countyA == countyB or countyA > countyB:
            continue
        
        df = data_rent[data_rent['Location'].isin({ countyA, countyB }) ]
        
        # print(df['Location'].unique())

        fig = px.line(df, x="Quarter", y="VALUE", color='Property Type', line_dash='Location', line_dash_sequence=[None, 'dash'], title="RTB Average Monthly Rent - {} vs {}".format(countyA, countyB), width=graph_width, height=graph_height, color_discrete_sequence=px.colors.qualitative.Vivid)

        fig.update_layout(
            xaxis = dict(
                tickmode = 'array',
                tickangle = -45
            ),
            showlegend = True,
            margin=dict(l=0, r=0, t=50),
            font=dict(
                size=16,  # Set the font size here
            ),
            xaxis_title="Year (by quarter)", yaxis_title="Monthly Rent (€)",
        )
        
        # for trace in fig['data']:
        #     trace.showlegend = False
        #     # fig.legend.title.text = ''
            
        # fig['layout'].legend.title.text = "County"
        # add_grouped_legend(fig, df, 'Quarter', dict(line_dash = 'Location'))
        
        fig.update_layout(legend=dict(
            orientation="v",
            # y=-0.65
        ))
        
        # fig.add_layout_image(
        # dict(
        #     source=legend,
        #     xref="paper", yref="paper",
        #     x=0, y=-0.45,
        #     sizex=1, sizey=1,
        #     # xanchor="left", yanchor="bottom"
        #     )
        # )
        
        fig.show()
        
        # fig.write_html("images/compare/{}_{}.html".format(countyA, countyB), include_plotlyjs=False, full_html=False)